In [120]:
import torch
import json
import numpy as np
import pandas as pd
import torch.nn as nn
import random
import evaluate
import ast
import math
from tqdm import tqdm
from collections import Counter
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from transformers import AutoModel, AutoTokenizer
from datasets import Dataset
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
from torchcrf import CRF

In [121]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

In [122]:
PATH = "C:\\Users\\Franco\\Desktop\\projetos\\bert"
# BERT_MODEL_PATH = f"{PATH}\\bert-large-portuguese-cased"
BERT_MODEL_PATH = f"{PATH}\\bertimbau_dapt_final_nongrouped"
# BERT_MODEL_PATH = f"{PATH}\\bertimbau_dapt_final_grouped"
MODEL_NAME = "models/absa_nlstm_ncrf_3concat.pt"

# a grande maioria dos reviews não passam de 300 tokens, então isso agiliza o treinamento
MAX_LENGTH = 300
K = 5
NUM_EPOCHS = 4
USE_LSTM = False
USE_CRF = False
CONCAT_LAST_N_BERT_LAYERS = 3
TRAIN_BATCH_SIZE = 8
RUN_CROSS_VALIDATION = False
LSTM_HIDDEN = 256
DROPOUT = 0.3
BERT_DROPOUT = 0.1

In [123]:
class ABSA_LSTM_CRF(nn.Module):
    def __init__(
        self,
        bert_model_name,
        num_labels=7,
        bert_dropout=0.1,
        dropout=0.3,
        use_lstm=True,
        use_crf=True,
        concat_last_n_bert_layers=1,
        lstm_hidden=256
    ):
        super().__init__()
        
        self.use_lstm = use_lstm
        self.use_crf = use_crf
        self.num_labels = num_labels

        # 1. BERT
        self.bert = AutoModel.from_pretrained(
            bert_model_name,
            output_hidden_states=True,
        )

        # Dropout menor para BERT
        self.bert_dropout = nn.Dropout(bert_dropout)

        # Dropout maior para outras camadas
        self.dropout = nn.Dropout(dropout)

        # Definindo o tamanho da entrada que vem do BERT
        self.concat_last_n_bert_layers = concat_last_n_bert_layers
        bert_output_dim = self.bert.config.hidden_size * concat_last_n_bert_layers

        # 2. LSTM (Opcional)
        if self.use_lstm:
            self.lstm = nn.LSTM(
                input_size=bert_output_dim,
                hidden_size=lstm_hidden // 2,
                num_layers=1,
                bidirectional=True,
                batch_first=True
            )
            # A saída da Bi-LSTM será (hidden // 2) * 2 = lstm_hidden
            self._init_lstm_weights()
            classifier_input_dim = lstm_hidden
        else:
            # Se não tiver LSTM, o classificador recebe direto do BERT
            classifier_input_dim = bert_output_dim

        # 3. Classificador Final (Projeta para o número de labels)
        self.classifier = nn.Linear(classifier_input_dim, num_labels)
        
        # Inicialização de pesos do classificador
        nn.init.xavier_uniform_(self.classifier.weight)
        if self.classifier.bias is not None:
            nn.init.zeros_(self.classifier.bias)

        # 4. CRF (Opcional)
        if self.use_crf:
            self.crf = CRF(num_labels, batch_first=True)

    def _init_lstm_weights(self):
        for name, param in self.lstm.named_parameters():
            if "weight_ih" in name:
                nn.init.xavier_uniform_(param)
            elif "weight_hh" in name:
                nn.init.orthogonal_(param)
            elif "bias" in name:
                nn.init.zeros_(param)
                # Forget gate bias = 1
                n = param.size(0)
                param.data[n//4:n//2].fill_(1.0)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Concatenar as últimas N camadas do BERT conforme artigo original
        hidden_states = outputs.hidden_states
        token_repr = torch.cat(hidden_states[-self.concat_last_n_bert_layers:], dim=-1)

        x = self.bert_dropout(token_repr)

        if self.use_lstm:
            x, _ = self.lstm(x)
            x = self.dropout(x)

        emissions = self.classifier(x)
        return emissions

    def compute_loss(self, emissions, labels, attention_mask):
        if self.use_crf:
            safe_labels = labels.clone()
            safe_labels[safe_labels == -100] = 0

            loss = -self.crf(
                emissions,
                safe_labels,
                mask=attention_mask.bool(),
                reduction='mean'
            )
            return loss

        loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
        return loss_fct(
            emissions.view(-1, self.num_labels),
            labels.view(-1)
        )


    def decode(self, input_ids, attention_mask):
        if self.use_crf:
            emissions = self.forward(input_ids, attention_mask)

            predictions = self.crf.decode(
                emissions,
                mask=attention_mask.bool()
            )
            return predictions
        logits = self.forward(input_ids, attention_mask)
        predictions = torch.argmax(logits, dim=2)
        return predictions.detach().cpu().numpy().tolist()


In [124]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_PATH)
seqeval = evaluate.load("seqeval")

print(f'Using device: {device}')

Using device: cuda


In [125]:
ID2LABEL = {
    0: "O",
    1: "B-POS",
    2: "I-POS",
    3: "B-NEG",
    4: "I-NEG",
    5: "B-NEU",
    6: "I-NEU"
}

LABEL2ID = {
    "O": 0,
    "B-POS": 1,
    "I-POS": 2,
    "B-NEG": 3,
    "I-NEG": 4,
    "B-NEU": 5,
    "I-NEU": 6
}

PAD_LABEL_ID = -100

RELEVANT_LABELS = ["B-POS", "I-POS", "B-NEG", "I-NEG", "B-NEU", "I-NEU", "POS", "NEG", "NEU"]

In [126]:
# Este é um JSON criado manualmente com base na HOntology (que é a origem dos aspectos do dataset)
# Ele serve para "normalizar" os aspectos
# Ex: 'custo-benefício': ['custo-beneficio', 'custo beneficio', 'custo benefício']
# Como os textos do dataset possuem variações de escrita dos aspectos, esse dicionário ajuda a mapear todas para a classe esperada
with open('data/aspectos_use.json', encoding='utf-8') as f:
    data = json.load(f)
    replace_dict = {k: v for d in data for k, v in d.items()}


In [127]:
def group_reviews(df):
    df["review_id"] = df.groupby("review").ngroup().apply(lambda x: f"R{x+1}")
    grouped = defaultdict(lambda: {"review": "", "aspects": []})

    for _, row in df.iterrows():
        rid = row["review_id"]
        grouped[rid]["review"] = row["review"]
        grouped[rid]["aspects"].append({
            "start": row["start_position"],
            "end": row["end_position"],
            "polarity": row["polarity"],
            "term": row["aspect"],
        })

    return list(grouped.values())

In [128]:
def create_bio_tags_for_bert(item: dict) -> dict:
    if not tokenizer:
        raise ImportError("O tokenizador do BERTimbau não foi carregado.")

    text = item['review']
    entities = item['aspects']  # Lista de entidades no formato {'start', 'end', 'polarity'}

    # Tokeniza o texto e obtém os mapeamentos de offset
    encoding = tokenizer(text, return_offsets_mapping=True, truncation=True, max_length=MAX_LENGTH, padding='max_length').to(device)
    tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
    offset_mapping = encoding['offset_mapping']
    # word_ids = encoding.word_ids()

    polarity_map = {-1: "NEG", 1: "POS", 0: "NEU"}
    tags = [LABEL2ID['O']] * len(tokens) # Inicializa com o ID para 'O'
    
    for entity in sorted(entities, key=lambda x: x['start']):
        start_char = entity['start']
        end_char = entity['end']
        polarity_label = polarity_map.get(entity['polarity'], "UNK")

        is_first_token_in_entity = True
        for i, (start_offset, end_offset) in enumerate(offset_mapping):
            # Ignora tokens especiais como [CLS] e [SEP] que tem offset (0, 0)
            if start_offset == end_offset:
                tags[i] = PAD_LABEL_ID  # Marca tokens especiais com PAD_LABEL_ID (ignora no loss)
                continue

            # Se o span do subword cruza com o span da entidade
            if max(start_offset, start_char) < min(end_offset, end_char):
                if is_first_token_in_entity:
                    tag_string = f"B-{polarity_label}"
                    is_first_token_in_entity = False
                else:
                    tag_string = f"I-{polarity_label}"
                tags[i] = LABEL2ID.get(tag_string, LABEL2ID['O'])

    # Comentado a parte que atribui -100 para subwords (algo semelhante a uma lematização)
    # previous_word_id = None
    # for i in range(len(tags)):
    #     if tags[i] == PAD_LABEL_ID:
    #         continue
    #     current_word_id = word_ids[i]
    #     if current_word_id is None or current_word_id == previous_word_id:
    #         tags[i] = PAD_LABEL_ID

    #     previous_word_id = word_ids[i]
        
    return {
        "input_ids": encoding['input_ids'],
        "labels": tags,
        "attention_mask": encoding['attention_mask'],
    }

In [129]:
data = pd.read_csv('data/train2024.csv', sep=";", index_col=0)
grouped_data = group_reviews(data)
dataset = Dataset.from_list(grouped_data)
dataset = dataset.map(create_bio_tags_for_bert)

# Divisão entre treino, validação e teste
# Essa divisão foi feita para testar diferentes abordagens de pré-processamento e arquiteturas
# splits_1 = dataset.train_test_split(test_size=0.2, seed=42)
# train_dataset = splits_1['train']
# test_valid_dataset = splits_1['test']
# splits_2 = test_valid_dataset.train_test_split(test_size=0.5, seed=42)

# eval_dataset = splits_2['train']
# test_dataset = splits_2['test']

# # Verificando os tamanhos
# print(f"Treino: {len(train_dataset)}")
# print(f"Validação: {len(eval_dataset)}")
# print(f"Teste: {len(test_dataset)}")


Map:   0%|          | 0/1322 [00:00<?, ? examples/s]

In [130]:
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(x["input_ids"], dtype=torch.long) for x in batch])
    attention_mask = torch.stack([torch.tensor(x["attention_mask"], dtype=torch.long) for x in batch])
    labels = torch.stack([torch.tensor(x["labels"], dtype=torch.long) for x in batch])

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

full_loader = DataLoader(dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# usados nos testes iniciais apenas
# train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
# eval_loader = DataLoader(eval_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [131]:
def compute_metrics(predictions, labels):
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [ID2LABEL[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [ID2LABEL[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [132]:
kf = KFold(n_splits=K, shuffle=True, random_state=42)
fold_metrics = []

if RUN_CROSS_VALIDATION:
    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f"\n========== Fold {fold+1}/{K} ==========")
        train_subset = Subset(dataset, train_idx)
        val_subset   = Subset(dataset, val_idx)

        train_loader = DataLoader(
            train_subset,
            batch_size=TRAIN_BATCH_SIZE,
            shuffle=True,
            collate_fn=collate_fn
        )

        val_loader = DataLoader(
            val_subset,
            batch_size=TRAIN_BATCH_SIZE,
            shuffle=False,
            collate_fn=collate_fn
        )

        model = ABSA_LSTM_CRF(
            bert_model_name=BERT_MODEL_PATH,
            num_labels=len(ID2LABEL),
            use_lstm=USE_LSTM,
            use_crf=USE_CRF,
            dropout=DROPOUT,
            bert_dropout=BERT_DROPOUT,
            lstm_hidden=LSTM_HIDDEN,
            concat_last_n_bert_layers=CONCAT_LAST_N_BERT_LAYERS
        ).to(device)

        loss_fct = torch.nn.CrossEntropyLoss(ignore_index=PAD_LABEL_ID)

        optimizer = torch.optim.AdamW([
            {"params": model.bert.parameters(), "lr": 1e-5},
            {"params": model.classifier.parameters(), "lr": 3e-4},
        ])

        if model.use_lstm:
            optimizer = torch.optim.AdamW([
                {"params": model.bert.parameters(), "lr": 1e-5},
                {"params": model.lstm.parameters(), "lr": 1e-4},
                {"params": model.classifier.parameters(), "lr": 3e-4},
            ])

        # ===== TREINO =====
        for epoch in range(NUM_EPOCHS):
            model.train()
            total_loss = 0

            for batch in train_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                optimizer.zero_grad()

                logits = model(input_ids, attention_mask)

                loss = loss_fct(
                    logits.view(-1, logits.size(-1)),
                    labels.view(-1)
                )

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

                total_loss += loss.item()

            print(f"Epoch {epoch+1} | Train loss: {total_loss / len(train_loader):.4f}")

        # ===== AVALIAÇÃO (SEQEVAL) =====
        model.eval()

        all_logits = []
        all_labels = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                logits = model(input_ids, attention_mask)

                all_logits.append(logits.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

        all_logits = np.concatenate(all_logits, axis=0)
        all_labels = np.concatenate(all_labels, axis=0)

        metrics = compute_metrics(all_logits, all_labels)
        fold_metrics.append(metrics)

        print(
            f"Fold {fold+1} | "
            f"F1: {metrics['f1']:.4f} | "
            f"P: {metrics['precision']:.4f} | "
            f"R: {metrics['recall']:.4f}"
        )

        del model
        torch.cuda.empty_cache()


In [133]:
if RUN_CROSS_VALIDATION:
    kf_df = pd.DataFrame(fold_metrics)

    # Adiciona coluna do fold
    kf_df.insert(0, "fold", np.arange(1, len(kf_df) + 1))
    kf_df.to_csv("kfold_seqeval_results.csv", index=False)

    print("\nResultados salvos em kfold_seqeval_results.csv")

    print(f"F1 médio: {kf_df['f1'].mean():.4f}")
    print(f"Desvio padrão: {kf_df['f1'].std():.4f}")

In [134]:
# Treinamento Final
model = ABSA_LSTM_CRF(
    bert_model_name=BERT_MODEL_PATH,
    num_labels=len(ID2LABEL),
    use_lstm=USE_LSTM,
    use_crf=USE_CRF,
    dropout=DROPOUT,
    bert_dropout=BERT_DROPOUT,
    lstm_hidden=LSTM_HIDDEN,
    concat_last_n_bert_layers=CONCAT_LAST_N_BERT_LAYERS
).to(device)

total_steps = len(full_loader) * NUM_EPOCHS
warmup_steps = int(total_steps * 0.1) 


loss_fct = nn.CrossEntropyLoss(ignore_index=PAD_LABEL_ID) 
optimizer = torch.optim.AdamW([
    {"params": model.bert.parameters(), "lr": 1e-5},
    {"params": model.classifier.parameters(), "lr": 3e-4},
])

if model.use_lstm:
    optimizer = torch.optim.AdamW([
        {"params": model.bert.parameters(), "lr": 1e-5},
        {"params": model.lstm.parameters(), "lr": 1e-4},
        {"params": model.classifier.parameters(), "lr": 3e-4},
    ])

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)


for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}")
    model.train()
    total_loss = 0

    for batch in full_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        logits = model(input_ids, attention_mask)

        loss = loss_fct(
            logits.view(-1, logits.size(-1)),
            labels.view(-1)
        )

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # recomendado
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    print(f"Train loss: {total_loss / len(full_loader):.4f}")

Some weights of BertModel were not initialized from the model checkpoint at C:\Users\Franco\Desktop\projetos\bert\bertimbau_dapt_final_nongrouped and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1
Train loss: 0.3907

Epoch 2
Train loss: 0.0656

Epoch 3
Train loss: 0.0509

Epoch 4
Train loss: 0.0417


In [135]:
torch.save(model.state_dict(), MODEL_NAME)


In [136]:
def process_term(term):
    term = term.strip().replace(" - ", "-")
    term = term.lower()
    for replace_to, replace_words in replace_dict.items():
        if term in replace_words:
            term = replace_to
            # print(f"Replaced '{term}' with '{replace_to}'")
    return term

In [137]:
def group_bi_tags_old(tag_list):
    result = []
    current_phrase = ""
    current_tag = None
    # previous_item = None
    for item in tag_list:
        if not isinstance(item, tuple) or len(item) != 2:
            print(f"Invalid item: {item}")
            continue  # Skip invalid entries

        word, tag = item
        if not isinstance(tag, str):
            print(f"Invalid tag type for item: {item}. Expected string, got {type(tag).__name__}.")
            # print(f"Item: {item}")
            # print(f"Previous item: {previous_item}")
            continue # Skip this item or handle it differently
    
        if tag.startswith("B-"):
            if current_phrase:
                term = process_term(current_phrase)
                result.append((term, current_tag))
            current_phrase = word
            current_tag = tag

        elif tag.startswith("I-") and current_tag and tag[2:] == current_tag[2:]:
            current_phrase += f" {word}"
        else:
            if current_phrase:
                term = process_term(current_phrase)
                result.append((term, current_tag))
            result.append((word, tag))
            current_phrase = ""
            current_tag = None
        
        # previous_item = item
    if current_phrase:
        term = process_term(current_phrase)
        result.append((term, current_tag))

    return result

In [138]:
LABEL2POLARITY = {
    "POS": 1,
    "NEG": -1,
    "NEU": 0,
    "B-POS": 1,
    "I-POS": 1,
    "B-NEG": -1,
    "I-NEG": -1,
    "B-NEU": 0,
    "I-NEU": 0,
} 

In [139]:
# Agrupa B's e I'
# Caso haja conflito de tags dentro de uma mesma entidade (ex: B-NEU + I-NEG), resolve pegando a tag mais frequente
# Remove tags "órfãs" (I- sem B- anterior)
# Trasforma tags em polaridade
def group_bi_tags(tag_list):
    result = []
    current_phrase = [] 
    current_tags_seen = [] 
    
    for item in tag_list:
        if not isinstance(item, tuple) or len(item) != 2: 
            continue
        word, tag = item
        
        # --- CASO 1: Início de Entidade (B-) ---
        if tag.startswith("B-"):
            # 1.a) Se já tinha algo aberto, fecha e salva
            if current_phrase:
                term = " ".join(current_phrase)
                # Resolve conflito (ex: B-NEU + I-NEG vira NEG)
                final_tag = Counter(current_tags_seen).most_common(1)[0][0]
                result.append((term, final_tag))
            
            # 1.b) Abre nova entidade
            current_phrase = [word]
            current_tags_seen = [tag[2:]] # Guarda 'NEU' sem o B-

        # --- CASO 2: Continuação Válida (I-) ---
        # Só entra aqui se for I- E se tivermos uma frase aberta
        elif tag.startswith("I-") and current_phrase:
            current_phrase.append(word)
            current_tags_seen.append(tag[2:]) # Guarda 'NEG' sem o I-

        # --- CASO 3: Resto (O, ou I- Órfão) ---
        else:
            # 3.a) Se tinha entidade aberta, fecha e salva ela antes de processar o atual
            if current_phrase:
                term = " ".join(current_phrase)
                term = process_term(term)
                final_tag = Counter(current_tags_seen).most_common(1)[0][0]
                result.append((term, final_tag))
                current_phrase = []
                current_tags_seen = []

            # 3.b) Processa a palavra atual
            # Se for órfão (I- perdido), forçamos virar 'O'. Se já for 'O', mantém.
            if tag.startswith("I-"):
                tag_corrigida = 'O' 
            else:
                tag_corrigida = tag
            
            # ADICIONA A PALAVRA ATUAL À LISTA (Isso evita ficar vazio)
            result.append((word, tag_corrigida))

    if current_phrase:
        term = " ".join(current_phrase)
        term = process_term(term)
        final_tag = Counter(current_tags_seen).most_common(1)[0][0]
        result.append((term, final_tag))

    return result

In [140]:
class ABSA_Predictor:
    def __init__(
        self,
        model_path,
        bert_model_name='neuralmind/bert-base-portuguese-cased',
        max_len=512,
        device='cuda',
        use_lstm=False,
        use_crf=False,
        lstm_hidden=256,
        dropout=0.3,
        bert_dropout=0.1,
        concat_last_n_bert_layers=4
    ):
        self.max_len = max_len
        self.use_crf = use_crf

        self.tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

        self.model = ABSA_LSTM_CRF(
            bert_model_name=bert_model_name,
            num_labels=len(ID2LABEL),
            use_lstm=use_lstm,
            use_crf=use_crf,
            lstm_hidden=lstm_hidden,
            dropout=dropout,
            bert_dropout=bert_dropout,
            concat_last_n_bert_layers=concat_last_n_bert_layers
        )

        state_dict = torch.load(model_path, map_location=device)
        self.model.load_state_dict(state_dict)

        self.model.to(device)
        self.model.eval()
        print("Modelo carregado")

    def predict(self, text):
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        with torch.no_grad():
            emissions = self.model(input_ids, attention_mask)

            if self.use_crf:
                predictions = self.model.crf.decode(
                    emissions,
                    mask=attention_mask.bool()
                )[0]  # batch = 1
            else:
                predictions = torch.argmax(emissions, dim=-1)[0].tolist()

        tokens = self.tokenizer.convert_ids_to_tokens(input_ids[0])

        result = []
        for i, tag_id in enumerate(predictions):
            if attention_mask[0, i] == 0:
                continue

            token = tokens[i]
            if token in ['[CLS]', '[SEP]']:
                continue

            label = ID2LABEL[tag_id]

            if token.startswith("##") and result:
                last_word, last_tag = result[-1]
                result[-1] = (last_word + token[2:], last_tag)
            else:
                result.append((token, label))

        return result


In [141]:
predictor = ABSA_Predictor(
    model_path=MODEL_NAME, 
    bert_model_name=BERT_MODEL_PATH,
    max_len=MAX_LENGTH,
    use_lstm=USE_LSTM,
    use_crf=USE_CRF,
    lstm_hidden=LSTM_HIDDEN,
    dropout=DROPOUT,
    bert_dropout=BERT_DROPOUT,
    concat_last_n_bert_layers=CONCAT_LAST_N_BERT_LAYERS
)

Some weights of BertModel were not initialized from the model checkpoint at C:\Users\Franco\Desktop\projetos\bert\bertimbau_dapt_final_nongrouped and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo carregado


In [142]:
def generate_results(csv_path, predictor, output_file="absa_results.csv", use_old_grouping=False, save_full_pred=False):    
    df = pd.read_csv(csv_path, sep=';')
    print(f"Avaliando {len(df)} reviews...")
    
    results = []

    for _, row in tqdm(df.iterrows(), total=len(df)):
        text = row['review']
        raw_pred = predictor.predict(text)
        
        gt = ast.literal_eval(row['aspects'])
        if use_old_grouping:
            grouped_pred = group_bi_tags_old(raw_pred)
        else:
            grouped_pred = group_bi_tags(raw_pred)

        relevant = [item for item in grouped_pred if item[1] in RELEVANT_LABELS]
        relevant = [(term, LABEL2POLARITY[tag]) for term, tag in relevant]

        result = {
            "preds": relevant,
            "labels": gt,
            "review": text,
        }

        if save_full_pred:
            result["full_pred"] = grouped_pred

        results.append(result)

    save_to = output_file
    if use_old_grouping:
        save_to = output_file.replace(".csv", "_oldgrouping.csv")
    pd.DataFrame(results).to_csv(save_to, sep=';', index=False)
    print(f"Resultados salvos em '{save_to}'")

In [143]:
# imagino que seria melhor fazer essa transformação no mesmo arquivo pra ficar mais fácil de comparar
dataset_teste = "data/task2_full_test_grouped.csv"

generate_results(dataset_teste, predictor)
# generate_results(dataset_teste, predictor, use_old_grouping=True)

Avaliando 282 reviews...


100%|██████████| 282/282 [00:20<00:00, 13.61it/s]

Resultados salvos em 'absa_results.csv'


In [144]:
# avaliação no dataset de teste da task 2 para testar Extração de Aspectos + Classificação de Sentimento
def evaluate_task2(df_name):
    df = pd.read_csv(f"{df_name}", sep=";")

    df['preds'] = df['preds'].apply(lambda x: ast.literal_eval(x))
    df['labels'] = df['labels'].apply(lambda x: ast.literal_eval(x))

    def key_of(x):
        if isinstance(x, (list, tuple)) and len(x) > 0:
            return (x[0], x[1])
        return x

    def to_multiset(seq):
        c = Counter()
        if seq is None or (isinstance(seq, float) and math.isnan(seq)):
            return c
        for x in seq:
            if x is None or (isinstance(x, float) and math.isnan(x)):
                continue
            k = key_of(x)
            if k is None:
                continue
            c[str(k).strip().lower()] += 1
        return c

    tp = fp = fn = 0
    total_gold = 0
    total_pred = 0

    for _, row in df.iterrows():
        gold = to_multiset(row["labels"])
        pred = to_multiset(row["preds"])

        row_tp = sum(min(gold[e], pred[e]) for e in gold)
        row_fp = sum(max(pred[e] - gold.get(e, 0), 0) for e in pred)
        row_fn = sum(max(gold[e] - pred.get(e, 0), 0) for e in gold)

        tp += row_tp
        fp += row_fp
        fn += row_fn
        total_gold += sum(gold.values())
        total_pred += sum(pred.values())

    # Micro (sobre todos os itens)
    micro_p = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    micro_r = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    micro_f1 = 2*micro_p*micro_r/(micro_p+micro_r) if (micro_p+micro_r) > 0 else 0.0

    print(f"TP: {tp} / itens-ouro: {total_gold}")
    print(f"FP: {fp} / itens-previstos: {total_pred}")
    print(f"FN: {fn}")
    print(f"[micro]  f1: {micro_f1:.3f} precisão: {micro_p:.3f}  recall: {micro_r:.3f}  ")
    print("===================")

    return micro_f1, micro_p, micro_r

In [145]:
f1, p, r = evaluate_task2("absa_results.csv")

with open("final_results_task2.txt", "a") as f:
    f.write(f"{MODEL_NAME}\t")
    f.write(f"F1: {f1:.4f}\t")
    f.write(f"P: {p:.4f}\t")
    f.write(f"R: {r:.4f}\n")
# evaluate_task2("absa_results_oldgrouping.csv")

TP: 850 / itens-ouro: 1176
FP: 316 / itens-previstos: 1166
FN: 326
[micro]  f1: 0.726 precisão: 0.729  recall: 0.723  
